<a href="https://colab.research.google.com/github/martinpius/ARCHITECTURES/blob/main/GOOGLE_NET_PYTORCH_IMPLEMENTATION_FROM_SCRATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on CoLaB with torch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)} {e}\n>>>> please correct {type(e)} and reload your drive")
  COLAB = False
def time_fmt(t: float = 123.98)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h}, min: {m:>02}, sec: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f">>>> testing the time formating function....\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are on CoLaB with torch version 1.9.0+cu102
>>>> testing the time formating function....
>>>> time elapsed	hrs: 0, min: 02, sec: 03.00


In [18]:
#In this notebook we are going to implement the GooGleNet from scratch:
import torch.nn as nn
import torch.functional as F

In [19]:
#Since the GooGleNet is formed by a combination of convolutional blocks with
#different number of filters (kernels) we build the network starting with a conv
#blocks 

In [21]:
class GoogleNet(nn.Module):
  def __init__(self, in_channels, num_classes = 1000):
    super(GoogleNet, self).__init__()
    #First block is only convolution and maxpool layers
    self.conv1 = ConvBlock(in_channels = in_channels, out_channels = 64, kernel_size = (7,7), stride = (2,2), padding = (3,3))
    self.conv2 = ConvBlock(in_channels = 64, out_channels = 192, kernel_size = (3,3), stride = (1,1), padding = (1,1))
    self.maxpool2 = nn.MaxPool2d(kernel_size = (3,3), stride = (2,2), padding = (1,1))
    self.maxpool1 = nn.MaxPool2d(kernel_size = (3,3), stride = (2,2), padding = (1,1))
    
    #The inception block start here (combination of several conv blocks with different kernels sizes)
    #inputs (arguments of the inception blocks) are copied from the inception paper
    self.inception3a = Inception_Block(192, 64, 96, 128, 16,32,32)
    self.inception3b = Inception_Block(256, 128,128,192,32,96,64)
    self.maxpool3 = nn.MaxPool2d(kernel_size = (3,3), stride = (2,2), padding = (1,1))
    self.inception4a = Inception_Block(480, 192, 96, 208, 16, 48, 64)
    self.inception4b = Inception_Block(512, 160, 112, 224, 24, 64, 64)
    self.inception4c = Inception_Block(512, 128, 128, 256, 24, 64, 64)
    self.inception4d = Inception_Block(512, 112, 114, 288, 32, 64, 64)
    self.inception4e = Inception_Block(528, 256, 160, 320, 32, 128,128)
    self.maxpool4 = nn.MaxPool2d(kernel_size = (3,3), stride = (2,2), padding = (1,1))
    self.inception5a = Inception_Block(832, 256, 160, 320, 32, 128, 128)
    self.inception5b = Inception_Block(832, 384, 192, 384, 48, 128, 128)
    self.avgpool = nn.AvgPool2d(kernel_size = (7,7), stride = (1,1))
    self.drp = nn.Dropout(p = 0.45)
    self.fc = nn.Linear(in_features = 1024, out_features = num_classes)
  
  def forward(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.inception3a(x)
    x = self.inception3b(x)
    x = self.maxpool3(x)
    x = self.inception4a(x)
    x = self.inception4b(x)
    x = self.inception4c(x)
    x = self.inception4d(x)
    x = self.inception4e(x)
    x = self.maxpool4(x)
    x = self.inception5a(x)
    x = self.inception5b(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.drp(x)
    return self.fc(x)

class Inception_Block(nn.Module):
  def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1max):
    super(Inception_Block, self).__init__()
    self.branch_1 = ConvBlock(in_channels = in_channels, out_channels = out_1x1, kernel_size = 1)
    self.branch_2 = nn.Sequential(
        ConvBlock(in_channels = in_channels, out_channels = red_3x3, kernel_size = 1),
        ConvBlock(in_channels = red_3x3, out_channels = out_3x3, kernel_size = 3, padding = 1))
    self.branch_3 = nn.Sequential(
        ConvBlock(in_channels = in_channels, out_channels = red_5x5, kernel_size = 1),
        ConvBlock(in_channels = red_5x5, out_channels = out_5x5, kernel_size = 5, padding = 2))
    self.branch_4 = nn.Sequential(
        nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1),
        ConvBlock(in_channels = in_channels, out_channels = out_1x1max, kernel_size = 1))
  
  def forward(self, input_tensor):
    x = torch.cat([self.branch_1(input_tensor), self.branch_2(input_tensor), self.branch_3(input_tensor), self.branch_4(input_tensor)], dim = 1)
    return x
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, **kwargs):
    super(ConvBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, **kwargs)
    self.relu = nn.ReLU()
    self.bnorm = nn.BatchNorm2d(num_features = out_channels)
  
  def forward(self, input_tensor):
    x = self.relu(self.conv(input_tensor))
    x = self.bnorm(x)
    return x


In [22]:
#Instantiate the model class and test if its gives the desired output:
rnd_data = torch.randn(size = (1, 3, 224, 224)).to(device = device)
model = GoogleNet(in_channels = 3, num_classes = 1000).to(device = device)
print(f">>>> the expected output shape is: {model(rnd_data).shape}")

>>>> the expected output shape is: torch.Size([1, 1000])


In [23]:
print(model)

GoogleNet(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu): ReLU()
    (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (bnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=1, ceil_mode=False)
  (maxpool1): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=1, ceil_mode=False)
  (inception3a): Inception_Block(
    (branch_1): ConvBlock(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU()
      (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch_2): Sequential(
      (0): ConvBlock(
        (conv): Conv2d(192, 96, kernel_size=(1, 1)